In [5]:
import subprocess
import json
from pathlib import Path
import pandas as pd

In [6]:
def run_pylint(file_path: Path):
    """
    Run pylint on a given Python file and return parsed JSON results.
    """
    cmd = ["pylint", "--output-format=json", str(file_path)]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    try:
        data = json.loads(result.stdout)
    except json.JSONDecodeError:
        print("Failed to parse pylint output.")
        data = []
    
    return data

In [7]:
# Create a simple buggy Python file for testing
sample_file = Path("sample_buggy.py")
sample_file.write_text("""
def add_numbers(a, b):
    return a + b

def unused_function(x):
    y = x * 2
    # Missing return statement
""")

111

In [8]:
results = run_pylint(sample_file)
print(f"Found {len(results)} issues")
results[:3]  # show first 3 issues


Found 4 issues


[{'type': 'convention',
  'module': 'sample_buggy',
  'obj': '',
  'line': 1,
  'column': 0,
  'endLine': None,
  'endColumn': None,
  'path': 'sample_buggy.py',
  'symbol': 'missing-module-docstring',
  'message': 'Missing module docstring',
  'message-id': 'C0114'},
 {'type': 'convention',
  'module': 'sample_buggy',
  'obj': 'add_numbers',
  'line': 2,
  'column': 0,
  'endLine': 2,
  'endColumn': 15,
  'path': 'sample_buggy.py',
  'symbol': 'missing-function-docstring',
  'message': 'Missing function or method docstring',
  'message-id': 'C0116'},
 {'type': 'convention',
  'module': 'sample_buggy',
  'obj': 'unused_function',
  'line': 5,
  'column': 0,
  'endLine': 5,
  'endColumn': 19,
  'path': 'sample_buggy.py',
  'symbol': 'missing-function-docstring',
  'message': 'Missing function or method docstring',
  'message-id': 'C0116'}]

In [9]:
df = pd.DataFrame(results)
df[["type", "message", "line", "symbol", "message-id"]].head(10)

,type,message,line,symbol,message-id
0,convention,Missing module docstring,1,missing-module-docstring,C0114
1,convention,Missing function or method docstring,2,missing-function-docstring,C0116
2,convention,Missing function or method docstring,5,missing-function-docstring,C0116
3,warning,Unused variable 'y',6,unused-variable,W0612


In [10]:
def run_bandit(file_path: Path):
    """
    Run bandit security scanner on a given Python file and return parsed JSON results.
    """
    cmd = ["bandit", "-f", "json", "-r", str(file_path)]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    try:
        data = json.loads(result.stdout)
    except json.JSONDecodeError:
        print("Failed to parse bandit output.")
        data = {}
    
    return data


In [11]:
bandit_results = run_bandit(sample_file)
print(f"Bandit found {len(bandit_results.get('results', []))} issues")
bandit_results.get("results", [])[:3]  # show first 3 issues


Bandit found 0 issues


[]

In [12]:
df_bandit = pd.DataFrame(bandit_results.get("results", []))
if not df_bandit.empty:
    df_bandit[["filename", "issue_severity", "issue_confidence", "issue_text"]].head(10)
else:
    print("No issues found by Bandit.")


No issues found by Bandit.


In [13]:
def merge_results(file_path: Path, pylint_data, bandit_data):
    # pylint aggregation
    pylint_msgs = [m for m in pylint_data if m.get("path") == str(file_path)]
    pylint_summary = {
        "pylint_total": len(pylint_msgs),
        "pylint_error": sum(1 for m in pylint_msgs if m.get("type") == "error"),
        "pylint_warning": sum(1 for m in pylint_msgs if m.get("type") == "warning"),
        "pylint_convention": sum(1 for m in pylint_msgs if m.get("type") == "convention"),
        "pylint_refactor": sum(1 for m in pylint_msgs if m.get("type") == "refactor"),
    }
    
    # bandit aggregation
    bandit_msgs = [r for r in bandit_data.get("results", []) if r.get("filename") == str(file_path)]
    severities = {"LOW": 0, "MEDIUM": 0, "HIGH": 0}
    for r in bandit_msgs:
        severities[r.get("issue_severity", "LOW")] += 1
    
    bandit_summary = {
        "bandit_total": len(bandit_msgs),
        "bandit_high": severities["HIGH"],
        "bandit_medium": severities["MEDIUM"],
        "bandit_low": severities["LOW"],
    }
    
    return {**pylint_summary, **bandit_summary}


In [14]:
merged = merge_results(sample_file, results, bandit_results)
print("Merged summary for sample_buggy.py:")
pd.DataFrame([merged])

Merged summary for sample_buggy.py:


,pylint_total,pylint_error,pylint_warning,pylint_convention,pylint_refactor,bandit_total,bandit_high,bandit_medium,bandit_low
0,4,0,1,3,0,0,0,0,0


In [15]:
# If you add more files to scan, loop through them here
files_to_scan = [sample_file]  # extend with Path("another_file.py"), etc.

records = []
for f in files_to_scan:
    pylint_data = run_pylint(f)
    bandit_data = run_bandit(f)
    summary = merge_results(f, pylint_data, bandit_data)
    records.append({"file": str(f), **summary})

df_combined = pd.DataFrame(records)
df_combined


,file,pylint_total,pylint_error,pylint_warning,pylint_convention,pylint_refactor,bandit_total,bandit_high,bandit_medium,bandit_low
0,sample_buggy.py,4,0,1,3,0,0,0,0,0
